<img src="img/mmselfsup_logo.png">

# 模型自监督预训练教程

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)

**作者**：OpenMMLab

## 0. 自监督预训练任务介绍：Masked Autoencoders (MAE)

**MAE 基本思想**：将输入图像分块 `patch`，随机遮挡住一部分图像块。将未被遮挡住的图像块和对应的位置信息一同输入到模型中，让模型恢复被遮挡的图像块内容。

<img src="img/MAE.png">

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMSelfSup 的依赖库：MMCV

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv-full

### 1.3  安装 MMSelfSup

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd /content/mmselfsup

In [ ]:
!git checkout v0.9.0

In [ ]:
# Install MMSelfSup from source
!pip install -e . 

### 1.4 检查安装是否正确

In [ ]:
import mmselfsup
print(mmselfsup.__version__)

## 2. 准备数据集

### 2.0 数据集介绍

本教程将在 `Tiny ImageNet` 数据集上训练 Masked Autoencoders (MAE) 模型。

Tiny ImageNet 数据集是 ImageNet 的一个子集。

该数据集包含 200 个类别，每个类别有 500 张训练图片、50 张验证图片和 50 张测试图片，共 120,000 张图像。每张图片均为 64×64 彩色图片。

数据集官方下载地址：http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.1 下载数据集

使用 GNU [Wget](https://www.gnu.org/software/wget/) 工具从斯坦福官方网站下载：http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:
!pwd

In [ ]:
!mkdir data
%cd data
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.2 解压

In [ ]:
!unzip -q tiny-imagenet-200.zip

In [ ]:
!rm -rf tiny-imagenet-200.zip

### 2.3 查看目录

In [ ]:
# Check data directory
!apt-get install tree
!tree -d /content/mmselfsup/data

### 2.4 准备标注文件

为了减少大家重写 `加载数据集` 代码的负担，我们整理好了标注文件，复制到数据集根目录 `mmselfsup/data/tiny-imagenet-200` 下即可。

In [ ]:
%cd /content
!git clone https://github.com/BizhuWu/MMSelfSup_tutorials
!cp /content/MMSelfSup_tutorials/anno_files/train.txt /content/mmselfsup/data/tiny-imagenet-200/train.txt

## 3. 编写模型自监督预训练的配置文件

In [ ]:
!pwd
%cd /content/mmselfsup

1. 新建一个名为 `mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py` 的配置文件。（配置文件命名要求 & 含义可参考[这里](https://mmsegmentation.readthedocs.io/zh_CN/latest/tutorials/config.html#id3))



2. 写训练配置文件的内容：
    1. 继承 [mae_vit-base-p16_8xb512-coslr-400e_in1k.py](https://github.com/open-mmlab/mmselfsup/blob/master/configs/selfsup/mae/mae_vit-base-p16_8xb512-coslr-400e_in1k.py) 配置文件
    2. 根据需求修改参数 samples_per_gpu（单个 GPU 的 Batch size）和 workers_per_gpu （单个 GPU 分配的数据加载线程数）
    3. 修改数据集路径和数据标注文件路径
    4. 根据 batch size 调整学习率（调整原则请参考：[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id2)）

In [ ]:
%%writefile /content/mmselfsup/configs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py

_base_ = 'mae_vit-base-p16_8xb512-coslr-400e_in1k.py'

# dataset
data = dict(
    samples_per_gpu=32, 
    workers_per_gpu=2,
    train=dict(
        data_source=dict(
            data_prefix='data/tiny-imagenet-200/train',
            ann_file='data/tiny-imagenet-200/train.txt',
        )
    )
)

# optimizer
optimizer = dict(
    lr=1.5e-4 * 4096 / 256 * (32 / 512 * 8),
)

runner = dict(max_epochs=1)

## 4. 模型自监督预训练

+ 使用 `单张/多张 GPU` 训练
```shell
sh tools/dist_train.sh ${CONFIG_FILE} ${GPUS} --work-dir ${YOUR_WORK_DIR} [optional arguments]
```
**注意**：我们推荐使用分布式训练工具 `tools/dist_train.sh` 来启动训练任务（即使您只用一张 GPU 进行训练）。
因为一些自监督预训练算法需要用多张 GPU 进行训练，为此 MMSelfSup 支持了多卡训练可能会用到的模块，如 `SyncBN` 等。如果算法在训练的过程中使用到了这些模块，但不使用分布式训练，就会报错。

    其他可选参数 `optional arguments` 可参考[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id3)。

In [ ]:
!pwd

In [ ]:
!bash tools/dist_train.sh \
configs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py \
1 \
--work_dir work_dirs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200/ 

## 5. 自监督预训练效果可视化 demo

## 6. 自监督预训练模型 T-SNE 图片向量可视化